In [1]:
from GUI import GUI

In [2]:
gui = GUI(img_file='data/input/2.jpg')

In [15]:
gui.element_detection(True, True, True)
gui.visualize_element_detection()

[Text Detection Completed in 1.512 s] Input: data/input/7.jpg Output: data/output\ocr\7.json
[Compo Detection Completed in 0.811 s] Input: data/input/7.jpg Output: data/output\ip\7.json
[Merge Completed] Input: data/input/7.jpg Output: data/output\uied\7.jpg


In [16]:
gui.layout_recognition()
gui.visualize_layout_recognition()

[Layout Recognition Completed in 1.513 s] Input: data/input/7.jpg Output: data/output\layout\7.json


In [3]:
gui.detect_element(False, False, True)
gui.visualize_element_detection()

[Merge Completed] Input: data/input/2.jpg Output: data/output\uied\2.jpg


In [4]:
gui.cvt_compos_json_to_dataframe()

In [5]:
gui.compos_df.recognize_element_groups_by_clustering()

In [8]:
gui.compos_df.compos_dataframe

,id,class,column_min,column_max,row_min,row_max,height,width,area,center,center_column,center_row,text_content,children,parent,alignment_in_group,gap,group
0,0,Compo,25,62,53,59,6,37,222,"(43.5, 56.0)",43.5,56.0,-1,-1,-1,v,-1,-1
1,1,Compo,401,425,56,81,25,24,600,"(413.0, 68.5)",413.0,68.5,-1,-1,-1,-1,-1,-1
2,2,Compo,25,54,64,69,5,29,145,"(39.5, 66.5)",39.5,66.5,-1,-1,-1,h,-1,nt-4
3,3,Compo,25,46,73,78,5,21,105,"(35.5, 75.5)",35.5,75.5,-1,-1,-1,h,-21,nt-4
4,4,Compo,45,62,123,139,16,17,272,"(53.5, 131.0)",53.5,131.0,-1,-1,-1,h,154,nt-3
5,5,Compo,216,233,125,139,14,17,238,"(224.5, 132.0)",224.5,132.0,-1,-1,-1,h,148,nt-3
6,6,Compo,381,408,122,140,18,27,486,"(394.5, 131.0)",394.5,131.0,-1,-1,-1,h,-1,nt-3
7,7,Compo,24,98,173,247,74,74,5476,"(61.0, 210.0)",61.0,210.0,-1,-1,-1,v,14,nt-1
8,8,Compo,391,429,219,238,19,38,722,"(410.0, 228.5)",410.0,228.5,-1,-1,-1,v,69,nt-0
9,9,Compo,351,364,223,236,13,13,169,"(357.5, 229.5)",357.5,229.5,-1,-1,-1,h,-1,-1
